In [2]:
from langchain_community.chat_models import ChatTongyi
from dotenv import load_dotenv

load_dotenv()

chat = ChatTongyi()

# PydanticOutputParser

In [3]:
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator


# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="开启一个笑话的问题")
    punchline: str = Field(description="解答笑话的答案")

    # You can add custom validation logic easily with Pydantic.
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "？":
            raise ValueError("Badly formed question!")
        return field


# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="根据用户的输入进行解答.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# And a query intended to prompt a language model to populate the data structure.
chain = prompt | chat | parser
chain.invoke({"query": "讲一个笑话"})

Joke(setup='为什么数学书总是很忧郁？', punchline='因为它有太多的问题。')

In [4]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"setup": {"title": "Setup", "description": "\\u5f00\\u542f\\u4e00\\u4e2a\\u7b11\\u8bdd\\u7684\\u95ee\\u9898", "type": "string"}, "punchline": {"title": "Punchline", "description": "\\u89e3\\u7b54\\u7b11\\u8bdd\\u7684\\u7b54\\u6848", "type": "string"}}, "required": ["setup", "punchline"]}\n```'

In [12]:
chain = prompt | chat | parser
joke = chain.invoke({"query": "为什么她吃不胖？"})  # 直接得到Joke对象

print(type(joke))  # <class '__main__.Joke'>
print(joke.setup)   # 直接访问属性
print(joke.punchline)

<class '__main__.Joke'>
为什么她吃不胖？
因为她是'瘦'人。


In [16]:
from langchain_core.messages import AIMessage

partial_chain = prompt | chat
llm_response = partial_chain.invoke({"query": "为什么她吃不胖？"})

# 检查响应类型并相应处理
if isinstance(llm_response, AIMessage):
    raw_text = llm_response.content
    print("AIMessage内容:", raw_text)
    
    try:
        joke = parser.parse(raw_text)
        print("解析成功:", joke)
    except Exception as e:
        print("解析失败:", e)
        print("原始内容:", raw_text)
else:
    print("未知响应类型:", type(llm_response))
    print("响应内容:", llm_response)

AIMessage内容: {
  "setup": "为什么她吃不胖？",
  "punchline": "因为她吃的都是'胖'字。"
}
解析成功: setup='为什么她吃不胖？' punchline="因为她吃的都是'胖'字。"


In [15]:
joke

Joke(setup='为什么她吃不胖？', punchline='因为她吃的是空气。')

# Json解析器

In [17]:
from langchain_core.output_parsers import JsonOutputParser

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Joke)

chain = prompt | chat | parser

chain.invoke({"query": "讲一个笑话"})

{'setup': '为什么数学书总是很忧郁？', 'punchline': '因为它有太多的问题。'}

In [8]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"setup": {"title": "Setup", "description": "\\u5f00\\u542f\\u4e00\\u4e2a\\u7b11\\u8bdd\\u7684\\u95ee\\u9898", "type": "string"}, "punchline": {"title": "Punchline", "description": "\\u89e3\\u7b54\\u7b11\\u8bdd\\u7684\\u7b54\\u6848", "type": "string"}}, "required": ["setup", "punchline"]}\n```'

# 自定义解析器
## Runnable Lambdas and Generators

In [18]:
from typing import Iterable

from langchain_core.messages import AIMessage, AIMessageChunk

def parse(ai_message: AIMessage) -> str:
    """Parse the AI message."""
    return ai_message.content.swapcase()


chain = chat | parse
chain.invoke("hello")

'hELLO! hOW CAN i ASSIST YOU TODAY? 😊'

## Runnable Generators

In [ ]:
from langchain_core.runnables import RunnableGenerator


def streaming_parse(chunks: Iterable[AIMessageChunk]) -> Iterable[str]:
    for chunk in chunks:
        yield chunk.content.swapcase()

# 对于streaming func, LangChain无法自动识别这是流式处理函数
# 需要明确告诉它是Runnable
streaming_parse = RunnableGenerator(streaming_parse)

chain = chat | streaming_parse

for chunk in chain.stream("tell me about yourself in one sentence"):
    print(chunk, end="|", flush=True)

i| AM| q|WEN|, A LARGE-SCALE| LANGUAGE MODEL DEVELOPED BY| aLIBABA gROUP.||

In [20]:
def parse(ai_message: AIMessage) -> str:
    return ai_message.content.swapcase()

chain = chat | parse
result = chain.invoke("hello")  # 等待完整响应，然后一次性处理

In [21]:
result

'hELLO! hOW CAN i ASSIST YOU TODAY? 😊'

In [23]:
def streaming_parse(chunks: Iterable[AIMessageChunk]) -> Iterable[str]:
    for chunk in chunks:
        yield chunk.content.swapcase()  # 逐个实时处理

chain = chat | RunnableGenerator(streaming_parse)
# 实时处理，无需等待完整响应
for chunk in chain.stream("tell me about yourself in one sentence"):
    print(chunk, end="|", flush=True)

i|'M| AN| ai| ASSISTANT DESIGNED TO HELP| WITH INFORMATION, ANSWER| QUESTIONS, AND ENGAGE| IN CONVERSATIONS.||

In [25]:
def step1(chunks: Iterable[AIMessageChunk]) -> Iterable[str]:
    for chunk in chunks:
        yield chunk.content.upper()

def step2(chunks: Iterable[str]) -> Iterable[str]:
    for chunk in chunks:
        yield chunk.replace(" ", "🔹")

def step3(chunks: Iterable[str]) -> Iterable[str]:
    for chunk in chunks:
        yield f"处理: {chunk}"

# 创建流式链
chain = (
    chat 
    | RunnableGenerator(step1) 
    | RunnableGenerator(step2)
    | RunnableGenerator(step3)
)

for chunk in chain.stream("tell me about yourself in one sentence"):
    print(chunk, end="|", flush=True)

处理: I|处理: 'M|处理: 🔹AN|处理: 🔹AI|处理: 🔹ASSISTANT🔹DESIGNED🔹TO🔹HELP|处理: 🔹WITH🔹INFORMATION,🔹ANSWER|处理: 🔹QUESTIONS,🔹AND🔹PROVIDE|处理: 🔹SUPPORT🔹IN🔹A🔹WIDE|处理: 🔹RANGE🔹OF🔹TOPICS.|处理: |

## 继承解析器基类

In [26]:
from langchain_core.exceptions import OutputParserException
from langchain_core.output_parsers import BaseOutputParser


# The [bool] desribes a parameterization of a generic.
# It's basically indicating what the return type of parse is
# in this case the return type is either True or False
class BooleanOutputParser(BaseOutputParser[bool]):
    """Custom boolean parser."""

    true_val: str = "YES"
    false_val: str = "NO"

    def parse(self, text: str) -> bool:
        cleaned_text = text.strip().upper()
        if cleaned_text not in (self.true_val.upper(), self.false_val.upper()):
            raise OutputParserException(
                f"BooleanOutputParser expected output value to either be "
                f"{self.true_val} or {self.false_val} (case-insensitive). "
                f"Received {cleaned_text}."
            )
        return cleaned_text == self.true_val.upper()

    @property
    def _type(self) -> str:
        return "boolean_output_parser"

In [27]:
parser = BooleanOutputParser()
chain = chat | parser
chain.invoke('回答我的问题，如果答案是"是"，直接回答"Yes"，如果答案是否，直接回答"No"。我的问题是："一周有7天吗"')

True

In [28]:
from langchain_core.prompts import PromptTemplate

# 创建提示词，要求LLM输出YES/NO
prompt = PromptTemplate(
    template="回答以下问题，只输出YES或NO:\n{question}",
    input_variables=["question"]
)

# 创建处理链
chain = prompt | chat | BooleanOutputParser()

# 调用
result = chain.invoke({"question": "天空是蓝色的吗？"})
print(result)  # True 或 False

True


In [29]:
# 使用中文响应
chinese_parser = BooleanOutputParser(true_val="是", false_val="否")

chain = prompt | chat | chinese_parser
result = chain.invoke({"question": "你喜欢Python吗？"})
print(result)

OutputParserException: BooleanOutputParser expected output value to either be 是 or 否 (case-insensitive). Received YES.

In [31]:
from langchain_core.prompts import PromptTemplate

# 明确要求中文输出
prompt = PromptTemplate(
    template="请用中文回答以下问题，只输出'是'或'否'，不要其他内容:\n{question}",
    input_variables=["question"]
)

chinese_parser = BooleanOutputParser(true_val="是", false_val="否")

chain = prompt | chat | chinese_parser
result = chain.invoke({"question": "你喜欢Python吗？"})
print(result)  # True 或 False

True


## 解析原始的LLM输出
模型的输出其实经常包含一些额外信息`metadata`的，因此如果解析器需要这部分信息的话，可以使用下面的方法

In [32]:
from typing import List

from langchain_core.exceptions import OutputParserException
from langchain_core.messages import AIMessage
from langchain_core.output_parsers import BaseGenerationOutputParser
from langchain_core.outputs import ChatGeneration, Generation


class StrInvertCase(BaseGenerationOutputParser[str]):
    """An example parser that inverts the case of the characters in the message.

    This is an example parse shown just for demonstration purposes and to keep
    the example as simple as possible.
    """

    def parse_result(self, result: List[Generation], *, partial: bool = False) -> str:
        """Parse a list of model Generations into a specific format.

        Args:
            result: A list of Generations to be parsed. The Generations are assumed
                to be different candidate outputs for a single model input.
                Many parsers assume that only a single generation is passed it in.
                We will assert for that
            partial: Whether to allow partial results. This is used for parsers
                     that support streaming
        """
        if len(result) != 1:
            raise NotImplementedError(
                "This output parser can only be used with a single generation."
            )
        generation = result[0]
        if not isinstance(generation, ChatGeneration):
            # Say that this one only works with chat generations
            raise OutputParserException(
                "This output parser can only be used with a chat generation."
            )
        return generation.message.content.swapcase()

In [33]:
chain = chat | StrInvertCase()
chain.invoke('回答我的问题，如果答案是"是"，直接回答"Yes"，如果答案是否，直接回答"No"。我的问题是："一周有7天吗"')

'yES'

# 类命名实体识别（NER）技术

In [34]:
# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="开启一个笑话的问题")
    punchline: str = Field(description="解答笑话的答案")

    # You can add custom validation logic easily with Pydantic.
    @validator("setup")
    # def question_ends_with_question_mark(cls, field):
    def test(cls, field):
        if field[-1] != "？":
            raise ValueError("Badly formed question!")
        return field


# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="根据用户的输入进行解答.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# And a query intended to prompt a language model to populate the data structure.
prompt_and_model = prompt | chat
output = prompt_and_model.invoke({"query": "讲一个笑话"})
parser.invoke(output)

Joke(setup='为什么数学书总是很忧郁？', punchline='因为它有太多的问题。')

In [38]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
# from langchain_openai import ChatOpenAI

output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()
print(format_instructions)


Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`
